# Spacy Span~~k~~Cat
![cat](spankcat.gif)

## Init settings
THere I created basic configuration for SpanCategorizer pipeline. I choose single-label variant since each span will have only one class assigned. Also, I set the language (ru) and make it use GPU.  

In [ ]:
!spacy init config spancat_config.cfg -l ru -p spancat_singlelabel -G -F

## Preprocessing data

In [1]:
!mkdir preporcessed_data

In [59]:
!python scripts/preprocess.py ../data/train.jsonl ./preporcessed_data/train.spacy ./preporcessed_data/val.spacy

514it [00:01, 319.05it/s]


I adjusted rules for spacy tokenizer, so it works with expected boundaries from train dataset. Also, I manually changed some boundaries that were incorrect (for example, missed the last character from the word) and dropped some examples from train set that were too hard to fix. See preprocess script for more details

## Training

I uploaded preprocessed data & config to Kaggle and continued there.

In [8]:
!pip uninstall -y cupy

Found existing installation: cupy 13.0.0
Uninstalling cupy-13.0.0:
  Successfully uninstalled cupy-13.0.0


In [14]:
!pip uninstall -y cupy-cuda12x

In [ ]:
!pip install -U spacy-transformers 'spacy[cuda12x]'

In [ ]:
!spacy train /kaggle/input/spacy-spancat-singlelabel-cfg/spancat_config.cfg -V -o ./spancat -g 0 --paths.train /kaggle/input/nlp-a3-dataset-in-spacy-format/train.spacy --paths.dev /kaggle/input/nlp-a3-dataset-in-spacy-format/val.spacy --nlp.batch_size 64 --training.logger.progress_bar true

[2024-04-21 23:27:12,490] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev', 'nlp.batch_size', 'training.logger.progress_bar']
ℹ Saving to output directory: spancat
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2024-04-21 23:27:13,856] [INFO] Set up nlp object from config
[2024-04-21 23:27:13,891] [DEBUG] Loading corpus from path: /kaggle/input/nlp-a3-dataset-in-spacy-format/val.spacy
[2024-04-21 23:27:13,896] [DEBUG] Loading corpus from path: /kaggle/input/nlp-a3-dataset-in-spacy-format/train.spacy
[2024-04-21 23:27:13,896] [INFO] Pipeline: ['transformer', 'spancat_singlelabel']
[2024-04-21 23:27:13,903] [INFO] Created vocabulary
[2024-04-21 23:27:13,903] [INFO] Finished initializing nlp object
[2024-04-21 23:27:24,544] [INFO] Initialized pipeline components: ['transformer', 'spancat_singlelabel']
✔ Initialized pipeline

============================= Training pipeline =============================
[2024-04-21 23:27:24,582] 

Training loop was failing to start sometimes, but it worked in the end. However, it died at around 40 epochs, so I downloaded the best model checkpoint and stopped the notebook.

Starting from the cell below, I returned to the local execution. 

In [10]:
!spacy benchmark accuracy model-best ./preprocessed_data/val.spacy

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-22 03:35:34.532063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 03:35:35.306377: W tensorflow/compiler/tf2tensorrt/utils/py_util

## Inference on dev set

In [11]:
!spacy apply model-best ../data/dev.jsonl predictions.spacy -tk sentences

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-22 03:39:04.042018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 03:39:04.730722: W tensorflow/compiler/tf2tensorrt/utils/py_util

In [45]:
import os
import warnings
import json

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

import spacy
from spacy import displacy
from spacy.tokens import DocBin

In [20]:
db = DocBin().from_disk("predictions.spacy")

In [23]:
docs = list(db.get_docs(spacy.blank("ru").vocab))

In [26]:
displacy.render(docs[0], style="span")

In [48]:
docs[0].spans["sc"][0].label_

'PROFESSION'

In [44]:
docs[0].spans["sc"][0].end_char

7

In [58]:
docs[0].text

'Генерал Д.Петреус назначен на пост главы ЦРУ.\n\nГенерал Дэвид Петреус назначен на пост главы Центрального разведывательного управления (ЦРУ). Такое решение сегодня принял Сенат США, передает Reuters.\n\nСообщается, что Д.Петреус избран главой ЦРУ подавляющим большинством членов Сената. Таким образом, генерал сменяет на этом посту Леона Панетту, который сегодня был назначен министром обороны США вместо ушедшего в отставку Роберта Гейтса. Д.Петреус приступит к исполнению своих обязанностей в сентябре 2011г.\n\nДо сегодняшнего назначения 58-летний Д.Петреус командовал контингентом НАТО в Афганистане. Теперь этот ставший вакантным пост займет генерал-лейтенант Джон Аллен.'

In [60]:
with open("../data/test.jsonl", "w") as out_file, open(
    "../data/dev.jsonl", "r"
) as test_file:
    for doc, test_line in zip(docs, test_file):
        test_data = json.loads(test_line)
        assert test_data["sentences"][:10] == doc.text[:10]
        print(
            json.dumps(
                {
                    "ners": [
                        [span.start_char, span.end_char - 1, span.label_]
                        for span in doc.spans["sc"]
                    ],
                    "id": test_data["id"],
                }
            ),
            file=out_file,
        )

Leaderboard score (dev.jsonl): 0.67

## Inference on test set

In [5]:
!spacy apply model-best ../data/test_set.jsonl predictions.spacy -tk sentences -F

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/eugen/coding/inno_labs/nlp-s24/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Loaded model model-best
65it [00:28,  2.32it/s]


In [6]:
import os
import warnings
import json

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

import spacy
from spacy.tokens import DocBin

In [7]:
db = DocBin().from_disk("predictions.spacy")
docs = list(db.get_docs(spacy.blank("ru").vocab))

In [8]:
with open("../data/test.jsonl", "w") as out_file, open(
    "../data/test_set.jsonl", "r"
) as test_file:
    for doc, test_line in zip(docs, test_file):
        test_data = json.loads(test_line)
        assert test_data["sentences"][:10] == doc.text[:10]
        print(
            json.dumps(
                {
                    "ners": [
                        [span.start_char, span.end_char - 1, span.label_]
                        for span in doc.spans["sc"]
                    ],
                    "id": test_data["id"],
                }
            ),
            file=out_file,
        )

Leaderboard score (test.jsonl): 0.73